In [1]:
# Reinforcement Learning
import numpy as np
import random

class MissionariesCannibalsEnv:
    def __init__(self):
        self.state = (3, 3, 1)  # Initial state (M_left, C_left, B_side) with boat on the right
        self.goal_state = (0, 0, 0)  # Goal state with all on the right bank and boat on the left

    def reset(self):
        self.state = (3, 3, 1)  # Reset to initial state
        return self.state

    def is_valid_state(self, m_left, c_left):
        # Check if the state is valid (missionaries not outnumbered by cannibals)
        return (m_left == 0 or m_left >= c_left) and (3 - m_left == 0 or 3 - m_left >= 3 - c_left)

    def step(self, action):
        m_left, c_left, b_side = self.state
        if b_side == 1:  # Boat on the right bank
            new_state = (m_left - action[0], c_left - action[1], 0)  # Move from right to left
        else:  # Boat on the left bank
            new_state = (m_left + action[0], c_left + action[1], 1)  # Move from left to right

        # Validate new state
        if self.is_valid_state(*new_state[:2]) and all(0 <= x <= 3 for x in new_state):
            reward = 1 if new_state == self.goal_state else -1  # Goal reward
            self.state = new_state
        else:
            reward = -10  # Invalid move
            new_state = self.state  # Stay in the same state

        return new_state, reward, new_state == self.goal_state

def train_q_learning(episodes, alpha, gamma, epsilon):
    env = MissionariesCannibalsEnv()
    q_table = np.zeros((4, 4, 2, 5))  # State space (M_left, C_left, B_side) x Action space

    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            m_left, c_left, b_side = state
            
            if random.uniform(0, 1) < epsilon:
                action = random.randint(0, 4)  # Explore
            else:
                action = np.argmax(q_table[m_left, c_left, b_side])  # Exploit
            
            action_values = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]
            
            if action < len(action_values):  # Ensure action is within valid range
                next_state, reward, done = env.step(action_values[action])
                next_m_left, next_c_left, next_b_side = next_state

                # Check that next state indices are valid before accessing q_table
                if 0 <= next_m_left < 4 and 0 <= next_c_left < 4 and 0 <= next_b_side < 2:
                    # Update Q-value
                    q_table[m_left, c_left, b_side, action] += alpha * (
                        reward + gamma * np.max(q_table[next_m_left, next_c_left, next_b_side]) - q_table[m_left, c_left, b_side, action]
                    )
                    state = next_state  # Update current state
                else:
                    print("Invalid next state:", next_state)  # Optional: Debugging info

    return q_table

# Parameters
episodes = 10000
alpha = 0.1
gamma = 0.9
epsilon = 0.1

q_table = train_q_learning(episodes, alpha, gamma, epsilon)

# Test the trained Q-table
def test_q_learning(q_table):
    env = MissionariesCannibalsEnv()
    state = env.reset()
    done = False
    path = []

    while not done:
        m_left, c_left, b_side = state
        action = np.argmax(q_table[m_left, c_left, b_side])
        action_values = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]
        
        if action < len(action_values):  # Ensure action is valid
            path.append(action_values[action])
            state, _, done = env.step(action_values[action])

    return path

solution_path = test_q_learning(q_table)
print("Solution Path:", solution_path)


Solution Path: [(0, 2), (0, 1), (0, 2), (0, 1), (2, 0), (1, 1), (2, 0), (0, 1), (0, 2), (1, 0), (1, 1)]


In [26]:
# A* search
import heapq

class MissionariesCannibalsEnv:
    def __init__(self):
        self.goal_state = (0, 0, 0)  # Goal: All on the right bank, boat on the left side

    def valid_state(self, state):
        m_left, c_left, _ = state
        m_right = 3 - m_left
        c_right = 3 - c_left

        # Valid if no missionaries are outnumbered on either side
        return (0 <= m_left <= 3 and 0 <= c_left <= 3 and
                (m_left == 0 or m_left >= c_left) and
                (m_right == 0 or m_right >= c_right))

    def get_successors(self, state):
        m_left, c_left, b_side = state
        successors = []
        possible_actions = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]

        for m_move, c_move in possible_actions:
            if b_side == 1:  # Boat on the right
                new_state = (m_left - m_move, c_left - c_move, 0)  # Move to the left bank
            else:  # Boat on the left
                new_state = (m_left + m_move, c_left + c_move, 1)  # Move to the right bank

            if self.valid_state(new_state):
                successors.append((new_state, (m_move, c_move)))
                
        return successors

    def heuristic(self, state):
        """ Heuristic function to estimate remaining steps to goal. """
        m_left, c_left, b_side = state
        return (m_left + c_left + 1) // 2  # Estimate crossings needed

    def a_star_search(self):
        initial_state = (3, 3, 1)  # Boat starts on the right
        # Priority queue for A* with (estimated total cost, cost so far, state, path)
        frontier = [(self.heuristic(initial_state), 0, initial_state, [])]
        visited = set()

        while frontier:
            est_total_cost, cost_so_far, state, path = heapq.heappop(frontier)

            if state == self.goal_state:
                return path  # Return the path to reach the goal

            if state in visited:
                continue

            visited.add(state)

            for successor, action in self.get_successors(state):
                if successor not in visited:
                    new_cost = cost_so_far + 1
                    est_cost = new_cost + self.heuristic(successor)
                    heapq.heappush(frontier, (est_cost, new_cost, successor, path + [action]))

        return None  # No solution found

# Initialize environment and find solution using A*
env = MissionariesCannibalsEnv()
solution_path = env.a_star_search()
print("Shortest Solution Path with A*:", solution_path)


Shortest Solution Path with A*: [(0, 2), (0, 1), (0, 2), (0, 1), (2, 0), (1, 1), (2, 0), (0, 1), (0, 2), (0, 1), (0, 2)]


In [27]:
# Markov Decision Process
import numpy as np

# Define the MDP environment for Missionaries and Cannibals
class MissionariesCannibalsMDP:
    def __init__(self):
        self.states = [(m, c, b) for m in range(4) for c in range(4) for b in range(2)]
        self.goal_state = (0, 0, 0)  # Goal: All people on the right, boat on the left side
        self.actions = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]  # Possible moves

    def is_valid(self, state):
        m, c, _ = state
        m_right = 3 - m
        c_right = 3 - c
        # Check validity of state: no missionaries outnumbered by cannibals
        return (0 <= m <= 3 and 0 <= c <= 3 and
                (m == 0 or m >= c) and
                (m_right == 0 or m_right >= c_right))

    def next_state(self, state, action):
        m, c, b = state
        m_move, c_move = action

        if b == 1:  # Boat is on the right
            new_state = (m - m_move, c - c_move, 0)  # Move to the left bank
        else:  # Boat is on the left
            new_state = (m + m_move, c + c_move, 1)  # Move to the right bank

        if self.is_valid(new_state):
            return new_state
        else:
            return None  # Invalid state

    def reward(self, state):
        return 1 if state == self.goal_state else -1

    def value_iteration(self, gamma=0.9, theta=1e-4):
        # Initialize value table with zeros
        V = {state: 0 for state in self.states}
        
        while True:
            delta = 0
            for state in self.states:
                if state == self.goal_state:
                    continue  # Skip the goal state

                best_action_value = float('-inf')
                for action in self.actions:
                    next_state = self.next_state(state, action)
                    if next_state is not None:
                        action_value = self.reward(next_state) + gamma * V[next_state]
                        best_action_value = max(best_action_value, action_value)

                delta = max(delta, abs(V[state] - best_action_value))
                V[state] = best_action_value

            if delta < theta:
                break

        # Extract policy from the value function
        policy = {}
        for state in self.states:
            if state == self.goal_state:
                continue

            best_action = None
            best_action_value = float('-inf')
            for action in self.actions:
                next_state = self.next_state(state, action)
                if next_state is not None:
                    action_value = self.reward(next_state) + gamma * V[next_state]
                    if action_value > best_action_value:
                        best_action_value = action_value
                        best_action = action

            policy[state] = best_action

        return policy

    def get_solution_path(self, policy):
        # Follow the policy to extract the path from start to goal state
        state = (3, 3, 1)  # Initial state
        path = []

        while state != self.goal_state:
            action = policy.get(state)
            if action is None:
                break  # No policy for this state

            path.append(action)
            state = self.next_state(state, action)

        return path

# Initialize the MDP and solve it with Value Iteration
mdp = MissionariesCannibalsMDP()
policy = mdp.value_iteration()
solution_path = mdp.get_solution_path(policy)
print("Optimal Solution Path with MDP:", solution_path)


Optimal Solution Path with MDP: [(0, 2), (0, 1), (0, 2), (0, 1), (2, 0), (1, 1), (2, 0), (0, 1), (0, 2), (1, 0), (1, 1)]


In [11]:
# Breadth First Search
from collections import deque

def is_valid_state(m, c, b):
    """ Check if the current state is valid. """
    return (m >= 0 and c >= 0 and
            (m == 0 or m >= c) and  # Left bank
            (M - m == 0 or M - m >= C - c))  # Right bank

def bfs_solution():
    """ Solve the problem using BFS and return moves. """
    initial_state = (3, 3, 1)  # (missionaries_left, cannibals_left, boat_position)
    goal_state = (0, 0, 0)      # All on the right bank
    queue = deque([(initial_state, [], [])])  # (current_state, path, moves)
    visited = set()
    
    while queue:
        current_state, path, moves = queue.popleft()
        m_left, c_left, boat_position = current_state
        
        if current_state in visited:
            continue
        
        visited.add(current_state)
        path.append(current_state)

        # Check if we've reached the goal state
        if current_state == goal_state:
            return path, moves
        
        # Possible moves: (missionaries, cannibals)
        for m_move in range(0, 3):
            for c_move in range(0, 3):
                if (m_move + c_move >= 1 and m_move + c_move <= 2):  # At least one and at most two can cross
                    # Determine new state after the move
                    if boat_position == 1:  # Boat is on the left
                        new_m_left = m_left - m_move
                        new_c_left = c_left - c_move
                        new_boat_position = 0
                        # Record the move as (left_to_right, right_to_left)
                        move = (m_move, c_move)
                    else:  # Boat is on the right
                        new_m_left = m_left + m_move
                        new_c_left = c_left + c_move
                        new_boat_position = 1
                        move = (0, m_move + c_move)  # Return move

                    new_state = (new_m_left, new_c_left, new_boat_position)

                    # Check if the new state is valid
                    if is_valid_state(new_m_left, new_c_left, new_boat_position):
                        queue.append((new_state, path.copy(), moves + [move]))  # Add the new state and move

    return [], []  # Return empty if no solution found

# Global variables
M, C = 3, 3  # Number of missionaries and cannibals

solution_path, solution_moves = bfs_solution()
for state in solution_path:
    print(state)


(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(1, 1, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)


In [7]:
# Depth First Search
class State:
    def __init__(self, missionaries, cannibals, boat):
        self.missionaries = missionaries
        self.cannibals = cannibals
        self.boat = boat  # 0 for left side, 1 for right side

    def is_valid(self):
        # Ensure no side has more cannibals than missionaries
        if self.missionaries < 0 or self.cannibals < 0 or self.missionaries > 3 or self.cannibals > 3:
            return False
        if (self.missionaries > 0 and self.missionaries < self.cannibals) or \
           (self.missionaries < 3 and self.cannibals < self.missionaries):
            return False
        return True

    def is_goal(self):
        return self.missionaries == 0 and self.cannibals == 0 and self.boat == 0

    def __hash__(self):
        return hash((self.missionaries, self.cannibals, self.boat))

    def __eq__(self, other):
        return (self.missionaries, self.cannibals, self.boat) == (other.missionaries, other.cannibals, other.boat)

    def __str__(self):
        return f"({self.missionaries}, {self.cannibals}, {self.boat})"


def dfs(state, path, visited):
    if state.is_goal():
        return path

    visited.add(state)

    # Possible moves: (M, C)
    possible_moves = [
        (1, 0), (0, 1), (2, 0), (0, 2), (1, 1)
    ]

    for m, c in possible_moves:
        if state.boat == 1:  # If the boat is on the right side
            new_state = State(state.missionaries - m, state.cannibals - c, 0)
        else:  # If the boat is on the left side
            new_state = State(state.missionaries + m, state.cannibals + c, 1)

        if new_state.is_valid() and new_state not in visited:
            result = dfs(new_state, path + [new_state], visited)
            if result:
                return result

    return None  # No solution found


def solve_missionaries_and_cannibals():
    initial_state = State(3, 3, 1)  # 3 missionaries, 3 cannibals, boat on right
    visited = set()
    path = [initial_state]

    result = dfs(initial_state, path, visited)

    if result:
        for state in result:
            print(state)
    else:
        print("No solution found.")


# Run the solution
solve_missionaries_and_cannibals()


(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(1, 1, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(1, 1, 1)
(0, 0, 0)


In [9]:
# Bi-Directional Search
from collections import deque 

class State:
    def __init__(self, missionaries_left, cannibals_left, boat_position):
        self.missionaries_left = missionaries_left
        self.cannibals_left = cannibals_left
        self.boat_position = boat_position  # 0 for left, 1 for right

    def is_goal(self):
        return self.missionaries_left == 0 and self.cannibals_left == 0 and self.boat_position == 0

    def is_valid(self):
        if self.missionaries_left < 0 or self.cannibals_left < 0:
            return False
        if self.missionaries_left > 3 or self.cannibals_left > 3:
            return False
        # Check the other side of the river
        missionaries_right = 3 - self.missionaries_left
        cannibals_right = 3 - self.cannibals_left
        if (self.missionaries_left > 0 and self.missionaries_left < self.cannibals_left) or \
           (missionaries_right > 0 and missionaries_right < cannibals_right):
            return False
        return True

    def get_successors(self):
        successors = []
        if self.boat_position == 1:  # Boat on the right side
            for m in range(3):
                for c in range(3):
                    if 1 <= m + c <= 2:  # At least one and at most two can cross
                        new_state = State(self.missionaries_left - m, self.cannibals_left - c, 0)
                        if new_state.is_valid():
                            successors.append(new_state)
        else:  # Boat on the left side
            for m in range(3):
                for c in range(3):
                    if 1 <= m + c <= 2:  # At least one and at most two can cross
                        new_state = State(self.missionaries_left + m, self.cannibals_left + c, 1)
                        if new_state.is_valid():
                            successors.append(new_state)
        return successors

    def __hash__(self):
        return hash((self.missionaries_left, self.cannibals_left, self.boat_position))

    def __eq__(self, other):
        return (self.missionaries_left, self.cannibals_left, self.boat_position) == (other.missionaries_left, other.cannibals_left, other.boat_position)

def bidirectional_search():
    initial_state = State(3, 3, 1)  # Start with boat on the right side
    goal_state = State(0, 0, 0)     # Goal: All on the right side, boat on the left

    front_queue = deque([initial_state])
    back_queue = deque([goal_state])

    visited_front = {initial_state: None}
    visited_back = {goal_state: None}

    while front_queue and back_queue:
        # Forward Search
        current = front_queue.popleft()
        for succ in current.get_successors():
            if succ not in visited_front:
                visited_front[succ] = current
                front_queue.append(succ)
                if succ in visited_back:
                    return reconstruct_path(visited_front, visited_back, succ)

        # Backward Search
        current = back_queue.popleft()
        for succ in current.get_successors():
            if succ not in visited_back:
                visited_back[succ] = current
                back_queue.append(succ)
                if succ in visited_front:
                    return reconstruct_path(visited_front, visited_back, succ)

    return None

def reconstruct_path(visited_front, visited_back, meeting_point):
    path = []
    # Reconstruct path from the start to the meeting point
    current = meeting_point
    while current is not None:
        path.append(current)
        current = visited_front[current]
    path.reverse()

    # Reconstruct path from the goal to the meeting point
    current = visited_back[meeting_point]
    while current is not None:
        path.append(current)
        current = visited_back[current]

    return path

# Test the bidirectional search
solution_path = bidirectional_search()
if solution_path:
    for state in solution_path:
        print(f"({state.missionaries_left}, {state.cannibals_left}, {state.boat_position})")
else:
    print("No solution found.")


(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(1, 1, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)
